<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

In [30]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

In [31]:
for layer in resnet.layers:
    layer.trainable = False

In [32]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [33]:
import skimage.io

In [34]:
from skimage.io import imread_collection
from skimage.transform import resize
from sklearn.model_selection import train_test_split


forests = 'data/forest/*.jpg'
mountains = 'data/mountain/*.jpg'
#creating a collection with the available images
forests = imread_collection(forests).concatenate()
mountains = imread_collection(mountains).concatenate()
y_0 =  np.zeros(forests.shape[0])
y_1 =  np.ones(mountains.shape[0])
X = np.concatenate([forests, mountains])
X = resize(X, (702,224,224,3))
y = np.concatenate([y_0, y_1])
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.5) 

In [26]:
x_train.shape

(351, 224, 224, 3)

In [29]:
y_train[:5]

array([0., 1., 0., 1., 0.])

## Instatiate Model

In [36]:
from tensorflow.keras.models import Sequential, Model # <- May Use
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [ ]:
'''
nmodel= Sequential()
for layer in model.layers:
    nmodel.add(layer)
'''

In [42]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
model = Model(resnet.input, predictions)
nmodel = Sequential()
nmodel.add(model)


nmodel.add(Conv2D(224, (3,3), activation='relu', input_shape=(224, 224, 3)))
nmodel.add(MaxPooling2D((2,2)))
nmodel.add(Conv2D(448, (3,3), activation='relu'))
nmodel.add(MaxPooling2D((2,2)))
nmodel.add(Conv2D(448, (3,3), activation='relu'))
nmodel.add(Flatten())
nmodel.add(Dense(448, activation='relu'))
nmodel.add(Dense(10, activation='softmax'))

nmodel.summary()

In [60]:
model.compile(optimizer='adam',
              #loss='sparse_categorical_crossentropy',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Fit Model

In [61]:
model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

Train on 351 samples, validate on 351 samples
351/351 [==============================] - 262s 747ms/sample - loss: 0.2463 - accuracy: 0.8917 - val_loss: 0.6877 - val_accuracy: 0.5043


In [62]:
#evaluate model
# I am going to change the loss metric and compare
#with binary cross entropy this is what I am getting:
#351/1 - 54s - loss: 0.7645 - accuracy: 0.5043

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

351/1 - 51s - loss: 0.6747 - accuracy: 0.5043


# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [ ]:
x_train, x_test = x_train / 255.0, x_test /255.0

In [63]:
x_train[0].shape

(224, 224, 3)

In [64]:
nmodel = Sequential()
#nmodel.add(model)


nmodel.add(Conv2D(224, (3,3), activation='relu', input_shape=(224, 224, 3)))
nmodel.add(MaxPooling2D((2,2)))
nmodel.add(Conv2D(448, (3,3), activation='relu'))
nmodel.add(MaxPooling2D((2,2)))
nmodel.add(Conv2D(448, (3,3), activation='relu'))
nmodel.add(Flatten())
nmodel.add(Dense(448, activation='relu'))
nmodel.add(Dense(224, activation='softmax'))

nmodel.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 222, 222, 224)     6272      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 224)     0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 109, 109, 448)     903616    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 448)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 52, 52, 448)       1806784   
_________________________________________________________________
flatten (Flatten)            (None, 1211392)           0         
_________________________________________________________________
dense_6 (Dense)              (None, 448)              

In [68]:
# Compile Model

model.compile(optimizer='adam',
              #loss='sparse_categorical_crossentropy',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [69]:
# Fit Model

model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

Train on 351 samples, validate on 351 samples
351/351 [==============================] - 261s 745ms/sample - loss: 0.0943 - accuracy: 0.9687 - val_loss: 0.6824 - val_accuracy: 0.6439


In [70]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

351/1 - 56s - loss: 0.6811 - accuracy: 0.6439


# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [ ]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language